In [433]:
import pandas as pd
import numpy as np
import requests
import warnings
from sklearn.model_selection import train_test_split,KFold
warnings.filterwarnings("ignore")
from sklearn.pipeline import Pipeline
from math import sqrt

In [436]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import train_test_split,KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.ensemble import VotingClassifier, ExtraTreesClassifier,RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC

In [3]:
barts_hotspots = pd.read_csv('data/barts_hotspots.csv')
barts_to_all = pd.read_csv('data/barts_to_all.csv')
hotspots_to_all = pd.read_csv('data/hotspots_to_all.csv')
hours_q1 = pd.read_csv('data/hours_q1.csv')
hours_q2 = pd.read_csv('data/hours_q2.csv')

In [161]:
barts_hotspots_filled.to_csv('barts_hotspots_updated.csv')

In [50]:
barts_hotspots.columns = ['Date',
 'Origin ID',
 'Origin Name',
 'Destination ID',
 'Destination Name',
 'Daily Mean Travel Time',
 'Daily Range - Lower',
 'Daily Range - Upper',
 'AM Mean Travel Time',
 'AM Range - Lower',
 'AM Range - Upper',
 'PM Mean Travel Time',
 'PM Range - Lower Bound Travel Time',
 'PM Range - Upper Bound Travel Time',
 'Midday Mean Travel Time',
 'Midday Range - Lower',
 'Midday Range - Upper',
 'Evening Mean Travel Time',
 'Evening Range - Lower',
 'Evening Range - Upper',
 'Early Morning Mean Travel Time',
 'Early Morning Range - Lower',
 'Early Morning Range - Upper',
 'weekday']

In [52]:
barts_hotspots.sort_values('Date')

,Date,Origin ID,Origin Name,Destination ID,Destination Name,Daily Mean Travel Time,Daily Range - Lower,Daily Range - Upper,AM Mean Travel Time,AM Range - Lower,...,Midday Mean Travel Time,Midday Range - Lower,Midday Range - Upper,Evening Mean Travel Time,Evening Range - Lower,Evening Range - Upper,Early Morning Mean Travel Time,Early Morning Range - Lower,Early Morning Range - Upper,weekday
2106,2019-01-01,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",3603,"Embarcadero, San Francisco, CA",684.0,381.0,1228.0,NaN,NaN,...,432.0,338.0,551.0,NaN,NaN,NaN,851.0,478.0,1515.0,1
2524,2019-01-01,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",3692,2nd Street and Stevenson Street (Montgomery BA...,483.0,319.0,729.0,NaN,NaN,...,464.0,343.0,627.0,427.0,322.0,565.0,493.0,312.0,777.0,1
2091,2019-01-01,3603,"Embarcadero, San Francisco, CA",3396,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...",1315.0,1074.0,1609.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2891,2019-01-01,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",3760,"Powell BART Station, Market St and Powell St, ...",677.0,453.0,1012.0,NaN,NaN,...,572.0,426.0,769.0,NaN,NaN,NaN,730.0,467.0,1139.0,1
2642,2019-01-01,3396,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...",3692,2nd Street and Stevenson Street (Montgomery BA...,1559.0,1233.0,1970.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1809.0,1551.0,2110.0,1
3005,2019-01-01,3396,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...",3760,"Powell BART Station, Market St and Powell St, ...",1576.0,1279.0,1942.0,NaN,NaN,...,1545.0,1248.0,1912.0,NaN,NaN,NaN,1685.0,1427.0,1989.0,1
2531,2019-01-01,3692,2nd Street and Stevenson Street (Montgomery BA...,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",517.0,274.0,973.0,NaN,NaN,...,396.0,248.0,630.0,311.0,184.0,525.0,571.0,297.0,1096.0,1
2672,2019-01-01,3394,"Fisherman's Wharf, San Francisco, CA",3692,2nd Street and Stevenson Street (Montgomery BA...,1125.0,756.0,1674.0,NaN,NaN,...,943.0,717.0,1240.0,761.0,631.0,918.0,1736.0,1011.0,2983.0,1
2078,2019-01-01,3394,"Fisherman's Wharf, San Francisco, CA",3603,"Embarcadero, San Francisco, CA",1124.0,670.0,1886.0,NaN,NaN,...,869.0,618.0,1221.0,643.0,508.0,814.0,1516.0,851.0,2699.0,1
1864,2019-01-01,3396,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...",3603,"Embarcadero, San Francisco, CA",1450.0,1115.0,1886.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [64]:
def stats(df):
    stats = df.describe().T
    stats['filled'] = stats['count']/len(df)
    return stats[['filled']]

In [160]:
#barts_hotspots_filled = barts_hotspots.copy()
barts_hotspots_filled['Destination Name'] = barts_hotspots_filled['Destination Name'].apply(lambda x: "Fisherman's Wharf, 286-298 Jefferson St, San Francisco, CA" if "Fisherman" in x else x)
barts_hotspots_filled['Destination Name'].nunique()
barts_hotspots_filled['Origin Name'] = barts_hotspots_filled['Origin Name'].apply(lambda x: "Fisherman's Wharf, 286-298 Jefferson St, San Francisco, CA" if "Fisherman" in x else x)
barts_hotspots_filled['Origin Name'].nunique()

6

In [85]:
def conditional_fill(df,col):
    df_filled = df.copy()
    if 'list' in str(type(col)):
        for i in col:
            df_filled[i] = barts_hotspots_filled.groupby(['Origin ID','weekday','Destination ID'])[i].transform(lambda x: x.fillna(x.median()))
    return df_filled

In [93]:
col_to_fill = list(barts_hotspots_filled.columns)
col_to_fill.remove('Date')
col_to_fill.remove('Origin ID')
col_to_fill.remove('Origin Name')
col_to_fill.remove('Destination ID')
col_to_fill.remove('Destination Name')
col_to_fill

['Daily Mean Travel Time',
 'Daily Range - Lower',
 'Daily Range - Upper',
 'AM Mean Travel Time',
 'AM Range - Lower',
 'AM Range - Upper',
 'PM Mean Travel Time',
 'PM Range - Lower Bound Travel Time',
 'PM Range - Upper Bound Travel Time',
 'Midday Mean Travel Time',
 'Midday Range - Lower',
 'Midday Range - Upper',
 'Evening Mean Travel Time',
 'Evening Range - Lower',
 'Evening Range - Upper',
 'Early Morning Mean Travel Time',
 'Early Morning Range - Lower',
 'Early Morning Range - Upper',
 'weekday']

In [94]:
barts_hotspots_filled = conditional_fill(barts_hotspots_filled,col_to_fill)

In [157]:
stats(barts_hotspots_filled)

,filled
Origin ID,1.000000
Destination ID,1.000000
Daily Mean Travel Time,1.000000
Daily Range - Lower,1.000000
Daily Range - Upper,1.000000
AM Mean Travel Time,0.857274
AM Range - Lower,0.857274
AM Range - Upper,0.857274
PM Mean Travel Time,0.960098
PM Range - Lower Bound Travel Time,0.960098


In [96]:
stats(barts_hotspots)

,filled
Origin ID,1.000000
Destination ID,1.000000
Daily Mean Travel Time,0.982198
Daily Range - Lower,0.982198
Daily Range - Upper,0.982198
AM Mean Travel Time,0.550031
AM Range - Lower,0.550031
AM Range - Upper,0.550031
PM Mean Travel Time,0.715163
PM Range - Lower Bound Travel Time,0.715163


In [109]:
barts_hotspots_filled.head()

,Date,Origin ID,Origin Name,Destination ID,Destination Name,Daily Mean Travel Time,Daily Range - Lower,Daily Range - Upper,AM Mean Travel Time,AM Range - Lower,...,Midday Mean Travel Time,Midday Range - Lower,Midday Range - Upper,Evening Mean Travel Time,Evening Range - Lower,Evening Range - Upper,Early Morning Mean Travel Time,Early Morning Range - Lower,Early Morning Range - Upper,weekday
0,2019-06-09,3396,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...",3603,"Embarcadero, San Francisco, CA",1588.0,1054.0,2392.0,NaN,NaN,...,1398.5,1191.5,1640.5,1149.5,861.0,1535.0,NaN,NaN,NaN,6
1,2019-04-28,3603,"Embarcadero, San Francisco, CA",3394,"Fisherman's Wharf, San Francisco, CA",639.0,512.0,796.0,579.0,427.0,...,654.0,524.0,817.0,608.0,492.0,753.0,NaN,NaN,NaN,6
2,2019-05-18,3603,"Embarcadero, San Francisco, CA",3394,"Fisherman's Wharf, San Francisco, CA",730.0,520.0,1024.0,567.0,460.0,...,791.0,590.0,1061.0,569.0,406.0,797.0,NaN,NaN,NaN,5
3,2019-04-18,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",3603,"Embarcadero, San Francisco, CA",512.0,371.0,705.0,648.0,461.0,...,528.0,398.0,702.0,442.0,321.0,608.0,363.0,297.0,443.0,3
4,2019-04-08,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",3603,"Embarcadero, San Francisco, CA",505.0,364.0,701.0,621.0,489.0,...,429.0,339.0,542.0,502.0,354.0,711.0,377.0,311.0,467.0,0


In [261]:
test_barts_hotspots_filled = barts_hotspots_filled[['Destination Name',"Origin Name",'Date','weekday','Midday Mean Travel Time','Evening Mean Travel Time','Early Morning Mean Travel Time']]

In [361]:
num = 200
test_barts_hotspots_filled_melt = pd.melt(test_barts_hotspots_filled, id_vars=['Destination Name',"Origin Name", 'Date','weekday'], 
        var_name="time_of_day", 
        value_name="Value")
test_barts_hotspots_filled_melt_sample = test_barts_hotspots_filled_melt.sample(num)
test_barts_hotspots_filled_melt_sample.head(num)

ValueError: a must be greater than 0 unless no samples are taken

In [255]:
def recommend(df, origin, destination, **argv):
    copy = df.copy()
    origin_filter = copy['Origin Name'].apply(lambda x: True if origin in x else False)
    destination_filter = copy['Destination Name'].apply(lambda x: True if destination in x else False)
    copy = copy[origin_filter&destination_filter]
    if not argv:
        return 'no arg'
    for key,value in argv.items():
        temp_filter = copy[key].apply(lambda x: True if str(value) in str(x) else False)
        copy = copy[temp_filter]
    output = copy.groupby('time_of_day')['Value'].mean().sort_values().index[0]
    return output

recommend(test_barts_hotspots_filled_melt_sample, 'Powell' , 'Fisherman', weekday=2)

'Early Morning Mean Travel Time'

In [265]:
test_barts_hotspots_filled_melt.groupby(['Destination Name','Origin Name','weekday','time_of_day'])['Value'].mean().to_json('sample.json')

In [252]:
barts_hotspots_filled[(barts_hotspots_filled['Date'] == "2019-05-19")& (barts_hotspots_filled['Origin Name'].apply(lambda x: True if 'Park' in x else False))]

,Date,Origin ID,Origin Name,Destination ID,Destination Name,Daily Mean Travel Time,Daily Range - Lower,Daily Range - Upper,AM Mean Travel Time,AM Range - Lower,...,Midday Mean Travel Time,Midday Range - Lower,Midday Range - Upper,Evening Mean Travel Time,Evening Range - Lower,Evening Range - Upper,Early Morning Mean Travel Time,Early Morning Range - Lower,Early Morning Range - Upper,weekday
372,2019-05-19,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",3603,"Embarcadero, San Francisco, CA",643.0,350.0,1180.0,380.5,296.5,...,947.0,489.0,1836.0,404.0,286.0,570.0,NaN,NaN,NaN,6
709,2019-05-19,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",3692,2nd Street and Stevenson Street (Montgomery BA...,1017.0,411.0,2513.0,401.5,289.0,...,1373.0,564.0,3339.0,441.0,244.0,795.0,431.0,238.0,781.0,6
1375,2019-05-19,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",3760,"Powell BART Station, Market St and Powell St, ...",1676.0,859.0,3267.0,NaN,NaN,...,2033.0,1406.0,2939.0,640.5,520.5,899.5,NaN,NaN,NaN,6


In [253]:
barts_hotspots_filled[(barts_hotspots_filled['Date'] == "2019-05-19")& (barts_hotspots_filled['Destination Name'].apply(lambda x: True if 'Park' in x else False))]

,Date,Origin ID,Origin Name,Destination ID,Destination Name,Daily Mean Travel Time,Daily Range - Lower,Daily Range - Upper,AM Mean Travel Time,AM Range - Lower,...,Midday Mean Travel Time,Midday Range - Lower,Midday Range - Upper,Evening Mean Travel Time,Evening Range - Lower,Evening Range - Upper,Early Morning Mean Travel Time,Early Morning Range - Lower,Early Morning Range - Upper,weekday
541,2019-05-19,3603,"Embarcadero, San Francisco, CA",3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",1066.0,520.0,2187.0,1728.0,1080.0,...,902.0,463.0,1757.0,538.0,380.0,762.0,557.0,368.0,842.0,6
791,2019-05-19,3692,"2nd Street and Stevenson Street, San Francisco...",3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",1325.0,489.0,3588.0,2632.0,1739.0,...,774.0,335.0,1789.0,525.0,340.0,810.0,454.0,306.0,676.0,6
1547,2019-05-19,3760,"Powell BART Station, Market St and Powell St, ...",3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",2142.0,959.0,4785.0,3223.0,2050.0,...,1183.0,639.0,2189.0,560.5,461.5,732.0,888.0,597.0,1321.0,6


In [318]:
test_df = pd.DataFrame([['startN','end1','week1','A',5],['start2','end1','week1','A',3],
             ['start2','end1','week1','A',1],
             ['start3','end1','week2','P',2],
             ['start2','end1','week1','P',3],
             ['qqq','end2','week2','A',3],
             ['start1','end1','week1','A',6],
            ['start2','end1','week1','P',3],
             ['start2','end2','week2','A',3],
             ['start1','end1','week1','A',6],
            ['start2','end2','week2','A',3],
             ['start1','end1','week1','A',2],
            ['qqq','end1','week2','P',22],
             ['start2','end2','week2','A',0],
            ['start3','end1','week1','P',3],
             ['start3','end2','week2','P',3],
             ['start1','end2','week2','P',3],
            ['start3','end2','week2','A',99],
             ['start1','end1','week1','P',2],
            ['start3','end1','week2','P',22],
             ['start2','end2','week2','A',0],
             ['qqq','end1','week1','P',6],
            ['start2','end1','week1','P',0],
             ['start1','end1','week2','A',3],
             ['start1','end2','week1','P',6],
             ['start3','end2','week1','A',9]])
test_df

,0,1,2,3,4
0,startN,end1,week1,A,5
1,start2,end1,week1,A,3
2,start2,end1,week1,A,1
3,start3,end1,week2,P,2
4,start2,end1,week1,P,3
5,qqq,end2,week2,A,3
6,start1,end1,week1,A,6
7,start2,end1,week1,P,3
8,start2,end2,week2,A,3
9,start1,end1,week1,A,6


In [319]:
pd.DataFrame(test_df.groupby([0,1,2,3])[4].apply(lambda x: x.mean())).sort_values(by=[0,1,4])

4
0      1    2     3           
qqq    end1 week1 P   6.000000
            week2 P  22.000000
       end2 week2 A   3.000000
start1 end1 week1 P   2.000000
            week2 A   3.000000
            week1 A   4.666667
       end2 week2 P   3.000000
            week1 P   6.000000
start2 end1 week1 A   2.000000
                  P   2.000000
       end2 week2 A   1.500000
start3 end1 week1 P   3.000000
            week2 P  12.000000
       end2 week2 P   3.000000
            week1 A   9.000000
            week2 A  99.000000
startN end1 week1 A   5.000000

In [374]:
test_df = pd.DataFrame(test_barts_hotspots_filled_melt.groupby(['Origin Name','Destination Name','weekday','time_of_day'])['Value'].mean()).reset_index() #.to_json('sample.json')


In [375]:
test_df.sort_values('Value').groupby(['Origin Name','Destination Name'])['Value'].transform('min')

Series([], Name: Value, dtype: float64)

In [359]:
output = test_df.sort_values('Value').drop_duplicates(['Origin Name','Destination Name'])
output

,Origin Name,Destination Name,weekday,time_of_day,Value
189,"Oracle Park, 24 Willie Mays Plaza, San Francis...",2nd Street and Stevenson Street (Montgomery BA...,0,Early Morning Mean Travel Time,347.520000
216,"Oracle Park, 24 Willie Mays Plaza, San Francis...","Embarcadero, San Francisco, CA",2,Early Morning Mean Travel Time,364.307692
24,"2nd Street and Stevenson Street, San Francisco...","Oracle Park, 24 Willie Mays Plaza, San Francis...",1,Early Morning Mean Travel Time,364.923077
99,"Embarcadero, San Francisco, CA","Oracle Park, 24 Willie Mays Plaza, San Francis...",5,Early Morning Mean Travel Time,407.000000
75,"Embarcadero, San Francisco, CA","Fisherman's Wharf, 286-298 Jefferson St, San F...",4,Early Morning Mean Travel Time,439.423077
0,"2nd Street and Stevenson Street, San Francisco...","Fisherman's Wharf, 286-298 Jefferson St, San F...",0,Early Morning Mean Travel Time,493.440000
246,"Oracle Park, 24 Willie Mays Plaza, San Francis...","Powell BART Station, Market St and Powell St, ...",5,Early Morning Mean Travel Time,511.000000
162,"Fisherman's Wharf, 286-298 Jefferson St, San F...","Embarcadero, San Francisco, CA",5,Early Morning Mean Travel Time,519.000000
292,"Powell BART Station, Market St and Powell St, ...","Oracle Park, 24 Willie Mays Plaza, San Francis...",6,Evening Mean Travel Time,569.730769
261,"Powell BART Station, Market St and Powell St, ...","Fisherman's Wharf, 286-298 Jefferson St, San F...",3,Early Morning Mean Travel Time,619.500000


In [368]:
import json
temp  = output.groupby('Origin Name')[['Destination Name','weekday','time_of_day','Value']].apply(lambda x: x.set_index('Destination Name').to_dict(orient='index')).to_dict()
with open('temp.json', 'w') as f:
    json.dump(temp, f,indent=4)

In [380]:
test_barts_hotspots_filled.columns = ['Destination_Name',
 'Origin_Name',
 'Date',
 'weekday',
 'Midday Mean Travel Time',
 'Evening Mean Travel Time',
 'Early Morning Mean Travel Time']

In [396]:
from collections import defaultdict

d = defaultdict(dict)

for row in test_barts_hotspots_filled.itertuples(index=False):
    d[row.Destination_Name][row.Origin_Name] = row.Date

pprint.pprint(d)

defaultdict(<class 'dict'>,
            {'2nd Street and Stevenson Street (Montgomery BART, San Francisco, CA': {"Fisherman's Wharf, 286-298 Jefferson St, San Francisco, CA": Timestamp('2019-03-13 00:00:00'),
                                                                                     'Oracle Park, 24 Willie Mays Plaza, San Francisco, CA': Timestamp('2019-01-02 00:00:00'),
                                                                                     'The Palace Of Fine Arts, 3601 Lyon St, San Francisco, CA': Timestamp('2019-03-01 00:00:00')},
             'Embarcadero, San Francisco, CA': {"Fisherman's Wharf, 286-298 Jefferson St, San Francisco, CA": Timestamp('2019-02-24 00:00:00'),
                                                'Oracle Park, 24 Willie Mays Plaza, San Francisco, CA': Timestamp('2019-02-09 00:00:00'),
                                                'The Palace Of Fine Arts, 3601 Lyon St, San Francisco, CA': Timestamp('2019-02-10 00:00:00')},
           

In [410]:
def retro_dictify(frame):
    d = {}
    for row in frame.values:
        here = d
        for elem in row[:-2]:
            if elem not in here:
                here[elem] = {}
            here = here[elem]
        here[row[-2]] = row[-1]
    return d
pprint.pprint(retro_dictify(test_barts_hotspots_filled[['Origin_Name','Destination_Name',
 'weekday',
 'Midday Mean Travel Time',
 'Evening Mean Travel Time',
 'Early Morning Mean Travel Time']]))

{'2nd Street and Stevenson Street, San Francisco, CA': {"Fisherman's Wharf, 286-298 Jefferson St, San Francisco, CA": {0: {627.0: {702.0: 481.0},
                                                                                                                           646.0: {630.0: 481.0},
                                                                                                                           652.0: {670.0: 481.0},
                                                                                                                           697.0: {652.0: 464.0},
                                                                                                                           703.0: {713.0: 481.0},
                                                                                                                           716.0: {571.0: 420.0,
                                                                                                                             

                                                                                                                     356.0: {387.0: 368.0,
                                                                                                                             471.0: 287.0},
                                                                                                                     361.0: {361.0: 320.0},
                                                                                                                     369.0: {314.0: 321.0},
                                                                                                                     371.0: {378.0: 237.0},
                                                                                                                     374.0: {495.0: 365.0},
                                                                                                                     381.0: {358.0: 405.0},
                     

                                                                                                                         1477.0: {1302.5: nan},
                                                                                                                         1565.0: {1302.5: nan},
                                                                                                                         1672.0: {1407.0: nan},
                                                                                                                         1813.0: {1302.5: nan}},
                                                                                                                     6: {1053.0: {nan: nan},
                                                                                                                         1094.0: {nan: nan},
                                                                                                                         1103.0: {nan: nan},


                                                                                                 506.0: {378.0: 361.0},
                                                                                                 513.0: {391.0: 372.0},
                                                                                                 534.0: {484.0: 349.0},
                                                                                                 539.0: {550.0: 472.0},
                                                                                                 562.0: {448.0: 391.0},
                                                                                                 566.0: {543.0: 427.0},
                                                                                                 586.0: {489.0: 415.0},
                                                                                                 902.0: {538.0: 557.0},
                                        

                                                                                                       695.0: {604.0: 702.0,
                                                                                                               620.0: 702.0},
                                                                                                       703.0: {549.0: 552.0},
                                                                                                       706.0: {637.0: 702.0},
                                                                                                       715.0: {703.0: 702.0},
                                                                                                       717.0: {686.0: 702.0},
                                                                                                       721.0: {698.0: 702.0,
                                                                                                               756.0: 70

                                                                                                                                        977.0: {994.0: 739.5},
                                                                                                                                        1002.0: {1055.0: 739.5},
                                                                                                                                        1007.0: {1072.0: 739.5},
                                                                                                                                        1015.0: {951.0: 739.5},
                                                                                                                                        1034.0: {985.0: 739.5},
                                                                                                                                        1036.0: {1083.0: 739.5},
                                      

                                                                                                                                      579.0: {438.0: 385.0},
                                                                                                                                      615.0: {368.0: 299.0},
                                                                                                                                      1373.0: {441.0: 431.0}}},
                                                          'Embarcadero, San Francisco, CA': {0: {360.0: {342.0: 377.0},
                                                                                                 412.0: {422.0: 285.0},
                                                                                                 417.0: {361.0: 377.0},
                                                                                                 429.0: {502.0: 377.0},
                                              

                                                                                                                                          848.0: nan,
                                                                                                                                          950.0: nan,
                                                                                                                                          1258.0: nan},
                                                                                                                                  729.0: {591.0: nan},
                                                                                                                                  868.0: {669.0: nan},
                                                                                                                                  939.0: {727.0: nan},
                                                                                               

                                                                                                                                  936.0: {560.5: 779.0},
                                                                                                                                  1145.0: {560.5: 779.0},
                                                                                                                                  1183.0: {560.5: 888.0},
                                                                                                                                  3427.0: {489.0: 779.0}}},
                                                                     'The Palace Of Fine Arts, 3601 Lyon St, San Francisco, CA': {0: {1035.0: {1208.0: nan},
                                                                                                                                      1305.0: {1208.0: nan},
                                                                     

                                                                                                                                      1485.0: {1647.5: nan},
                                                                                                                                      1495.0: {1647.5: nan},
                                                                                                                                      1504.0: {1647.5: nan},
                                                                                                                                      1639.0: {1647.5: nan},
                                                                                                                                      1785.0: {1647.5: nan}},
                                                                                                                                  6: {1156.0: {1398.0: nan},
                                                         

In [420]:
train_df = barts_hotspots_filled[['Destination Name',
 'Origin Name',
 'Date',
 'weekday',
 'Daily Mean Travel Time']]
train_df['month'] = train_df['Date'].dt.month

In [422]:
train_df.head()

,Destination Name,Origin Name,Date,weekday,Daily Mean Travel Time,month
0,"Embarcadero, San Francisco, CA","The Palace Of Fine Arts, 3601 Lyon St, San Fra...",2019-06-09,6,1588.0,6
1,"Fisherman's Wharf, 286-298 Jefferson St, San F...","Embarcadero, San Francisco, CA",2019-04-28,6,639.0,4
2,"Fisherman's Wharf, 286-298 Jefferson St, San F...","Embarcadero, San Francisco, CA",2019-05-18,5,730.0,5
3,"Embarcadero, San Francisco, CA","Oracle Park, 24 Willie Mays Plaza, San Francis...",2019-04-18,3,512.0,4
4,"Embarcadero, San Francisco, CA","Oracle Park, 24 Willie Mays Plaza, San Francis...",2019-04-08,0,505.0,4


In [439]:
def train(train_df):
    train_df = train_df.copy()
    X_train, X_test, y_train, y_test = train_test_split(train_df.drop(['Daily Mean Travel Time','Date'],axis=1), train_df['Daily Mean Travel Time'], test_size=0.25, random_state=0)
    
    col_c3 = ['Destination Name','Origin Name','weekday','month']
    c3_transformer = Pipeline(steps=[('onehot', OneHotEncoder())])

    preproc = ColumnTransformer(transformers=[
                                             ('c3', c3_transformer, col_c3)])

    pl = Pipeline(steps=[('preprocessor', preproc), ('regressor', LinearRegression())])
    pl.fit(X_train,y_train)
    pred = pl.predict(X_test)
    rms = sqrt(mean_squared_error(y_test, pred))
    preds = pl.predict(train_df.drop(['Daily Mean Travel Time','Date'],axis=1))
    train_df['pred'] = preds
    return rms,train_df

In [442]:
train(train_df)[1]

,Destination Name,Origin Name,Date,weekday,Daily Mean Travel Time,month,pred
0,"Embarcadero, San Francisco, CA","The Palace Of Fine Arts, 3601 Lyon St, San Fra...",2019-06-09,6,1588.0,6,1199.323525
1,"Fisherman's Wharf, 286-298 Jefferson St, San F...","Embarcadero, San Francisco, CA",2019-04-28,6,639.0,4,618.662783
2,"Fisherman's Wharf, 286-298 Jefferson St, San F...","Embarcadero, San Francisco, CA",2019-05-18,5,730.0,5,685.503096
3,"Embarcadero, San Francisco, CA","Oracle Park, 24 Willie Mays Plaza, San Francis...",2019-04-18,3,512.0,4,543.624105
4,"Embarcadero, San Francisco, CA","Oracle Park, 24 Willie Mays Plaza, San Francis...",2019-04-08,0,505.0,4,491.554427
5,"Embarcadero, San Francisco, CA","Oracle Park, 24 Willie Mays Plaza, San Francis...",2019-06-30,6,616.0,6,444.910882
6,"Fisherman's Wharf, 286-298 Jefferson St, San F...","Embarcadero, San Francisco, CA",2019-04-23,1,682.0,4,691.613354
7,"Fisherman's Wharf, 286-298 Jefferson St, San F...","Embarcadero, San Francisco, CA",2019-05-13,0,634.0,5,677.194603
8,"Embarcadero, San Francisco, CA","Fisherman's Wharf, 286-298 Jefferson St, San F...",2019-05-15,2,878.0,5,817.066754
9,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...","Embarcadero, San Francisco, CA",2019-04-14,6,996.0,4,1091.648035
